# **Facial Expression Recognition**
## **Ensemble Modeling**
### Week 6
### Alejandro Alemany, Sara Manrriquez, and Benjamin Zaretzky
<br/>
In this notebook we combine three of our prior models into an ensemble and evaluate their performance.

## Import Statements

In [ ]:
import numpy as np
import pandas as pd
import scikitplot
import matplotlib.pyplot as plt
from scipy import stats

from tensorflow import keras
from sklearn.metrics import classification_report

Since this competition is closed, we have access to the full dataset. Therefore, we will combine the private and public test data sets together. 

## Read in Full Data Set

In [ ]:
# Read in full data set
data = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')
data.columns = ['emotion', 'Usage', 'pixels']
print(data.shape)

In [ ]:
# View first five rows
data.head()

## Select Only Data in Test Sets

In [ ]:
# Select only rows that are in the public or private test set
test = data.loc[data["Usage"] != 'Training',['emotion','pixels']]
#test.drop(columns='Usage', inplace=True)
test.head()

## Reshape Pixels

In [ ]:
# Reshape the pixels
test['pixels'] = [np.fromstring(x, dtype=int, sep=' ').reshape(-1,48,48,1) for x in test['pixels']]

In [ ]:
# Combine pixels into single array
pixels = np.concatenate(test['pixels'].values)

print(pixels.shape)

In [ ]:
# Standardize the pixels values between 0 and 1
pixels = pixels / 255

## Load Trained Models

In [ ]:
model_1 = keras.models.load_model('../input/models/Facial Recognition Models/fer_v05_BZ.h5')
model_2 = keras.models.load_model('../input/models/Facial Recognition Models/fer_v04_BZ.h5')
model_3 = keras.models.load_model('../input/models/Facial Recognition Models/fer_v03_BZ.h5')
model_4 = keras.models.load_model('../input/models/Facial Recognition Models/fer_v02_BZ.h5')

## Generate Prediction Probabilities

In [ ]:
test_probs1 = model_1.predict(pixels)
test_probs2 = model_2.predict(pixels)
test_probs3 = model_3.predict(pixels)
test_probs4 = model_4.predict(pixels)

## Assign Each Sample a Predicted Label

In [ ]:
test_pred1 = np.argmax(test_probs1, axis=1)
test_pred2 = np.argmax(test_probs2, axis=1)
test_pred3 = np.argmax(test_probs3, axis=1)
test_pred4 = np.argmax(test_probs4, axis=1)

## Use Max Voting To Select Final Prediction

In [ ]:
final_pred = np.array([])
for i in range(0,len(test_pred1)):
    final_pred = np.append(final_pred, stats.mode([test_pred1[i], test_pred2[i], test_pred3[i], test_pred4[i]])[0])

## Combine Predicted Labels and Actual Labels

In [ ]:
test['predictions'] = np.round(final_pred,0)
test.head()

In [ ]:
emotion_cat = {0:'Anger', 1:'Disgust', 2:'Fear', 3:'Happiness', 4: 'Sadness', 5: 'Surprise', 6: 'Neutral'}
test['emotion'] = test['emotion'].apply(lambda x: emotion_cat[x])
test['predictions'] = test['predictions'].apply(lambda x: emotion_cat[x])

## Classification Report

In [ ]:
my_classification_report = classification_report(test['emotion'], test['predictions'])
print(my_classification_report)

## Number of Misclassified Samples

In [ ]:
print('Total Wrong Predictions:', np.sum(test['emotion'] != test['predictions']))

## Confusion Matrix

In [ ]:
scikitplot.metrics.plot_confusion_matrix(test['emotion'], test['predictions'], figsize=(7,7))    

## View Correctly and Incorrectly Classified Samples

### Correctly Classified

Below is a plot of correctly classified images. The blue text next to each image is the observed emotion and the red text is the predicted emotion.

In [ ]:
plt.close()
plt.rcParams["figure.figsize"] = [16,16]

row = 0
for emotion in np.unique(test['emotion'].values):
    all_emotion_images = test[(test['emotion'] == emotion) & (test['predictions'] == emotion)]
    for i in range(5):
        
        img = all_emotion_images.iloc[i,].pixels.reshape(48,48)
        actual_lab = emotion
        predicted_lab = all_emotion_images.iloc[i,].predictions

        plt.subplot(7,5,row+i+1)
        plt.imshow(img, cmap='binary_r')
        plt.text(-30, 5, s = str(actual_lab), fontsize=10, color='b')
        plt.text(-30, 10, s = str(predicted_lab), fontsize=10, color='r')
        plt.axis('off')
    row += 5
    
plt.show()

### Incorrectly Classified

Below is a plot of misclassified images. The blue text next to each image is the observed emotion and the red text is the predicted emotion.

In [ ]:
plt.close()
plt.rcParams["figure.figsize"] = [16,16]

row = 0
for emotion in np.unique(test['emotion'].values):
    all_emotion_images = test[(test['emotion'] == emotion) & (test['predictions'] != emotion)]
    for i in range(5):
        
        img = all_emotion_images.iloc[i,].pixels.reshape(48,48)
        actual_lab = emotion
        predicted_lab = all_emotion_images.iloc[i,].predictions

        plt.subplot(7,5,row+i+1)
        plt.imshow(img, cmap='binary_r')
        plt.text(-30, 5, s = str(actual_lab), fontsize=10, color='b')
        plt.text(-30, 10, s = str(predicted_lab), fontsize=10, color='r')
        plt.axis('off')
    row += 5
    
plt.show()